In [1]:
import pandas as pd
from datetime import datetime

Import weather data

In [25]:
file = f"../data/raw/weather_{datetime.today().strftime('%Y%m%d')}.csv"
modified_df = pd.read_csv(file)
original_df = modified_df.copy()
modified_df

,Unnamed: 0,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,hourly_units,hourly
0,time,52.52,13.419998,0.038862,0,GMT,GMT,38.0,iso8601,"['2025-08-16T00:00', '2025-08-16T01:00', '2025..."
1,temperature_2m,52.52,13.419998,0.038862,0,GMT,GMT,38.0,°C,"[21.6, 20.7, 19.9, 19.1, 18.5, 18.6, 18.9, 19...."


Extract hourly values

In [26]:
hourly_df = modified_df['hourly'].to_frame()
hourly_df

,hourly
0,"['2025-08-16T00:00', '2025-08-16T01:00', '2025..."
1,"[21.6, 20.7, 19.9, 19.1, 18.5, 18.6, 18.9, 19...."


Evaluate values, from string to list data type

In [ ]:
hourly_df['hourly'] = hourly_df['hourly'].apply(eval)
hourly_df

hourly    object
dtype: object

Switch columns with rows

In [13]:
transposed_df = hourly_df.transpose()
transposed_df

,0,1
hourly,"[2025-08-16T00:00, 2025-08-16T01:00, 2025-08-1...","[21.6, 20.7, 19.9, 19.1, 18.5, 18.6, 18.9, 19...."


Put each item from lists on separate row

In [14]:
exploded_df = transposed_df.explode(transposed_df.columns.to_list())
exploded_df

,0,1
hourly,2025-08-16T00:00,21.6
hourly,2025-08-16T01:00,20.7
hourly,2025-08-16T02:00,19.9
hourly,2025-08-16T03:00,19.1
hourly,2025-08-16T04:00,18.5
...,...,...
hourly,2025-08-22T19:00,17.9
hourly,2025-08-22T20:00,16.8
hourly,2025-08-22T21:00,15.8
hourly,2025-08-22T22:00,15.0


Remove and reset index

In [15]:
index_reset_df = exploded_df.reset_index(drop=True)
index_reset_df

,0,1
0,2025-08-16T00:00,21.6
1,2025-08-16T01:00,20.7
2,2025-08-16T02:00,19.9
3,2025-08-16T03:00,19.1
4,2025-08-16T04:00,18.5
...,...,...
163,2025-08-22T19:00,17.9
164,2025-08-22T20:00,16.8
165,2025-08-22T21:00,15.8
166,2025-08-22T22:00,15.0


Rename columns and add units

In [29]:
old_column_names = index_reset_df.columns.to_list()
new_column_names = modified_df[modified_df.columns[0]].to_list()
units = modified_df['hourly_units'].to_list()
col_dict = {}
for old, new, unit in zip(old_column_names, new_column_names, units):
    col_dict[old] = f"{new} ({unit})"
renamed_columns_df = index_reset_df.rename(columns=col_dict)
renamed_columns_df

,time (iso8601),temperature_2m (°C)
0,2025-08-16T00:00,21.6
1,2025-08-16T01:00,20.7
2,2025-08-16T02:00,19.9
3,2025-08-16T03:00,19.1
4,2025-08-16T04:00,18.5
...,...,...
163,2025-08-22T19:00,17.9
164,2025-08-22T20:00,16.8
165,2025-08-22T21:00,15.8
166,2025-08-22T22:00,15.0


Adjust column data types

In [ ]:
types_dict = {}
for col in renamed_columns_df.columns:
    types_dict[col] = 'datetime64[ns]' if 'time' in col.lower() else 'float64'
type_cast_df = renamed_columns_df.astype(types_dict)
type_cast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   time (iso8601)       168 non-null    datetime64[ns]
 1   temperature_2m (°C)  168 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.8 KB
